In [1]:
import pandas as pd

import sys
sys.path.append("src")

from merging_with_id import merging, create_id

In [3]:
# load datasets
datasets = {}
years = range(2018, 2021)
for year in years:
    print(f"Schools stats {year} Loading...")
    file_name = f'ZZSO-{year}.xlsx'
    dataset = pd.read_excel(f"./fin_data/{file_name}", header=1, dtype='str')
    datasets.update({year:dataset})
    print("success")

Schools stats 2018 Loading...
success
Schools stats 2019 Loading...
success
Schools stats 2020 Loading...
success


In [4]:
for year in years:
    print(year,'\n', list(datasets[year].columns))

2018 
 ['№ Області', 'Область', 'Тип територіальної одиниці', '№№', 'Назва тер. Одиниці', 'Найменування закладу', 'Ступінь закладу ( I, I-II, I-III)', 'Загальна площа приміщень школи ( без господарських будівель) (кв.м.)', 'Кількість штатних працівників педагогічного персоналу  разом', 'Кількість штатних працівників непедагогічного персоналу  разом', 'Кількість штатних працівників педагогічного персоналу  пенсійного віку', 'Питома вага непедагогічного персоналу у загальній кількості працівників, %**', 'Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %**', 'Кількість учнів разом', 'Кількість класів разом', 'Фактична наповнюваність класів, учнів на клас**', 'Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя**', 'Видатки на функціонування закладу (загальний фонд), тис.грн.\n2018рік (касові видатки)', 'Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (план)', 'Видатки на 1 учня, 2018, тис. 

In [5]:
rename_dict = {
    "Область": "regname",
    "Тип територіальної одиниці": "tertype",
    "Тип школи: міська / сільська": 'eotype',
    "Назва тер. Одиниці": "areaname",
    "Найменування закладу": "eoname",
    "Ступінь закладу ( I, I-II, I-III)": "eolevel",
    "Загальна площа приміщень школи ( без господарських будівель) (кв.м.)": "eosize",
    "Кількість штатних працівників педагогічного персоналу  разом": "teachstuff",
    "Кількість штатних працівників непедагогічного персоналу  разом": "nonteachstuff",
    "Кількість штатних працівників педагогічного персоналу  пенсійного віку": "teachstuffretage",
    "Кількість учнів разом": "pupils",
    "Кількість класів разом": "classes",
    "Разом кількість учнів, та дітей, які здобувають дошкільну освіту": "pupilswithpreschool",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2018рік (касові видатки)": "opex",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (касові видатки)": "opex",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (касові видатки)": "opex",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (план)": "opexplan",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)": "opexplan",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)": "opexplan",
    "Опорна": "hub",
    "Код бюджету": "budgetcode",
    "Форма власності": "ownership",
    "Код ДІСО": "DISO",
    "ЄДРПОУ": "EDRPOU"
    }

In [6]:
for year in years:
    datasets[year].rename(columns=rename_dict, inplace=True)
    to_drop = [col for col in datasets[year].columns if col not in rename_dict.values()]
    datasets[year].drop(columns = to_drop, inplace = True)
    datasets[year]['year'] = year


# Data cleaning 2020

## Location region

In [7]:
# load location dictionary
locations = pd.read_csv('./final_tables/locations.csv', encoding='utf-8', dtype=str)
locations

,regname,areaname,tername,KOATUU_2020,KATOTTG_2023
0,Запорізька область,Мелітопольський район,с.Терпіння,2323085101,UA23080270010078454
1,Хмельницька область,Красилівський район,м.Красилів,6822710100,UA68040210010032567
2,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,1223881501,UA12140170040016918
3,Чернівецька область,м.Чернівці,Шевченківський район міста,7310100000,UA73060610010033137
4,Миколаївська область,Врадіївський район,с.Кумарі,4822383001,UA48080050190079797
...,...,...,...,...,...
29214,США,м.Детройт,м.Детройт,0026010000,OC26010000000000000
29215,Велика Британія,м.Абериствіт,м.Абериствіт,0008030000,OC08030000000000000
29216,США,м.Нью-Йорк,м.Нью-Йорк,0026020000,OC26020000000000000
29217,США,м.Х'юстон,м.Х'юстон,0026040000,OC26040000000000000


In [8]:
datasets[2020].regname = datasets[2020].regname.apply(lambda x: 'м.Київ' if x == 'місто Київ' or x == 'м.Київ' else x+' область')
datasets[2020].regname.unique()

array(['Вінницька область', 'Волинська область',
       'Дніпропетровська область', 'Донецька область',
       'Житомирська область', 'Закарпатська область',
       'Запорізька область', 'Івано-Франківська область',
       'Київська область', 'Кіровоградська область', 'Луганська область',
       'Львівська область', 'Миколаївська область', 'Одеська область',
       'Полтавська область', 'Рівненська область', 'Сумська область',
       'Тернопільська область', 'Харківська область',
       'Херсонська область', 'Хмельницька область', 'Черкаська область',
       'Чернівецька область', 'Чернігівська область', 'м.Київ'],
      dtype=object)

In [9]:
locations['KOATUU_2020_reg'] = locations['KOATUU_2020'].str[:2]
locations

,regname,areaname,tername,KOATUU_2020,KATOTTG_2023,KOATUU_2020_reg
0,Запорізька область,Мелітопольський район,с.Терпіння,2323085101,UA23080270010078454,23
1,Хмельницька область,Красилівський район,м.Красилів,6822710100,UA68040210010032567,68
2,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,1223881501,UA12140170040016918,12
3,Чернівецька область,м.Чернівці,Шевченківський район міста,7310100000,UA73060610010033137,73
4,Миколаївська область,Врадіївський район,с.Кумарі,4822383001,UA48080050190079797,48
...,...,...,...,...,...,...
29214,США,м.Детройт,м.Детройт,0026010000,OC26010000000000000,00
29215,Велика Британія,м.Абериствіт,м.Абериствіт,0008030000,OC08030000000000000,00
29216,США,м.Нью-Йорк,м.Нью-Йорк,0026020000,OC26020000000000000,00
29217,США,м.Х'юстон,м.Х'юстон,0026040000,OC26040000000000000,00


In [10]:
loc = locations[['regname', 'KOATUU_2020_reg']]
loc.drop_duplicates(inplace= True)
loc.reset_index(drop=True, inplace=True)
loc

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_50960/3378581834.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc.drop_duplicates(inplace= True)


,regname,KOATUU_2020_reg
0,Запорізька область,23
1,Хмельницька область,68
2,Дніпропетровська область,12
3,Чернівецька область,73
4,Миколаївська область,48
5,Донецька область,14
6,Тернопільська область,61
7,Кіровоградська область,35
8,Одеська область,51
9,Львівська область,46


In [19]:
dataset_2020 = datasets[2020].merge(loc[['regname', 'KOATUU_2020_reg']], on=['regname'], how='left')
dataset_2020 .head()

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,classes,opex,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,9,4388.111,5006.502,NaN,02553000000,комунальна,7137,25844555,2020,05
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,12,6523.777,8614.601,NaN,02553000000,комунальна,6936,25844992,2020,05
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,21,9273.777,12627.197,NaN,02553000000,комунальна,7149,25844928,2020,05
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,19,8654.501,11029.117,NaN,02553000000,комунальна,7191,25844970,2020,05
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,31,14833.298,19559.764,NaN,02553000000,комунальна,6976,23101028,2020,05


In [20]:
set(dataset_2020.eolevel.unique())

{'0', 'I', 'I-II', 'I-III', 'I-IIІ', 'II', 'II-III', 'III', 'ІІ-ІІІ'}

In [21]:
values_to_drop = ['I', 'I-II', 'II']

dataset_2020  = dataset_2020[~dataset_2020.eolevel.isin(values_to_drop)]
dataset_2020.reset_index(drop=True, inplace=True)
dataset_2020 

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,classes,opex,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,9,4388.111,5006.502,NaN,02553000000,комунальна,7137,25844555,2020,05
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,12,6523.777,8614.601,NaN,02553000000,комунальна,6936,25844992,2020,05
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,21,9273.777,12627.197,NaN,02553000000,комунальна,7149,25844928,2020,05
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,19,8654.501,11029.117,NaN,02553000000,комунальна,7191,25844970,2020,05
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,31,14833.298,19559.764,NaN,02553000000,комунальна,6976,23101028,2020,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,30,19606.3,25731.415,NaN,26000000000,комунальна,22883,23392178,2020,80
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,34,22065.4,26469.2,NaN,26000000000,комунальна,23029,23392149,2020,80
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,18,15793.70295055802,10294.196,NaN,26000000000,комунальна,23280,26187881,2020,80
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,30,20353.2,13412.1,NaN,26000000000,комунальна,23154,22878447,2020,80


In [22]:
# check whether there are duplicates by name and location
dataset_2020[dataset_2020.duplicated(subset=['eoname', 'KOATUU_2020_reg'], keep=False)]

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,classes,opex,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg
6473,Тернопільська область,село,Бучацька міська ТГ,Озерянська загальноосвітня школа І-ІІІ ступенів,I-III,NaN,22,12,2,133,11,4379.4,5352.2,NaN,19555000000,комунальна,17857,24623237,2020,61
6610,Тернопільська область,село,Борщівська міська ТГ,Озерянська загальноосвітня школа І-ІІІ ступенів,I-III,NaN,25,16,1,196,11,6062.3,7561.6,NaN,19527000000,комунальна,17807,24620894,2020,61


In [23]:
dataset_2020['EDRPOU_checked'] = ''

In [24]:
dataset_2020.loc[dataset_2020.EDRPOU == '24623237', 'EDRPOU_checked'] = '24623237'
dataset_2020.loc[dataset_2020.EDRPOU == '24620894', 'EDRPOU_checked'] = '24620894'

In [25]:
dataset_2020[dataset_2020.EDRPOU.str.strip().str.len() != 8].EDRPOU.unique()

array(['2141288', '2141294', '2141302', '2141377', '62059', '7995',
       '2141383', '0', '2141408', '11', '2141354', '2141650', '4334672',
       '2142201', '2124769', '2133320', '2360228', '6669359', '6669419',
       '6669879', '6669750', '6669833', '6669856', '6669773', '6669827',
       '6670121', '6669744', '6669738', '6669786', '6670664', '6670635',
       '6670606', '6670753', '6670919', '6671155', '6671445', '6671267',
       '6671149', '6671497', '6671422', '6671468', '6671110', '6671474',
       '6671178', '6670374', '6670428', '6670405', '6670411', '6670380',
       '6670397', '6670368', '6670474', '6669263', '6669313', '6669655',
       '6669721', '6669394', '6669454', '6669253', '6669247', '6669336',
       '6671451', '6671439', '6671161', '6669804', '6669862', '6669649',
       '6669810', '6669603', '6669796', '6669840', '6670701', '6670776',
       '6670687', '6670693', '6670718', '6670552', '6670581', '6670598',
       '6670730', '6670782', '6670569', '6670575', '6670

In [26]:
dataset_2020.EDRPOU = dataset_2020.EDRPOU.apply(lambda x: x.zfill(8) if isinstance(x, str) and len(x) >= 6 else None)

In [27]:
dataset_2020[(dataset_2020.EDRPOU.isna())].eoname.unique()

array(['Турбівська загальноосвітня школа І-ІІІ ст. №2 _x000D_\nЛиповецької районної ради _x000D_\nВінницької області_x000D_\n_x000D_\n',
       'Середня загальноосвітня школа І - ІІІ ступенів села Уладівка Літинський район Вінницька область',
       'Середня загальноосвітня школа І-ІІІ ступенів с. Івча',
       'Комунальний заклад "Загальноосвітня школа І-ІІІ ступенів № 19 Вінницької міської ради"',
       'Старобільська загальноосвітня школа ІІ-ІІІ ступенів №4 Старобільської районної ради Луганської області'],
      dtype=object)

In [28]:
set_edrpou = {
    'Турбівська загальноосвітня школа І-ІІІ ст. №2 _x000D_\nЛиповецької районної ради _x000D_\nВінницької області_x000D_\n_x000D_\n': '21723820',
    'Середня загальноосвітня школа І - ІІІ ступенів села Уладівка Літинський район Вінницька область':'26229088',
    'Середня загальноосвітня школа І-ІІІ ступенів с. Івча':'26223660',
    'Комунальний заклад "Загальноосвітня школа І-ІІІ ступенів № 19 Вінницької міської ради"':'22103539',
    'Старобільська загальноосвітня школа ІІ-ІІІ ступенів №4 Старобільської районної ради Луганської області':'33199525'}

In [29]:
for school in set_edrpou:
    dataset_2020.loc[dataset_2020['eoname'] == school, 'EDRPOU'] = set_edrpou[school]
    dataset_2020.loc[dataset_2020['eoname'] == school, 'EDRPOU_checked'] = set_edrpou[school]

In [30]:
create_id(dataset_2020, 'eoname', 'id')

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,...,5006.502,NaN,02553000000,комунальна,7137,25844555,2020,05,,загальноосвтнйнавчальновиховнийкомплекс3ступшк...
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,...,8614.601,NaN,02553000000,комунальна,6936,25844992,2020,05,,кнззагальноосвтняшкола3ступ3козятинськавницькаобл
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,...,12627.197,NaN,02553000000,комунальна,7149,25844928,2020,05,,комунальнийнавчальнийзакладзагальноосвтняшкола...
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,...,11029.117,NaN,02553000000,комунальна,7191,25844970,2020,05,,загальноосвтнясередняшкола3ступ5козятинськавни...
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,...,19559.764,NaN,02553000000,комунальна,6976,23101028,2020,05,,комунальнийнавчальнийзакладспецалзованашкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,25731.415,NaN,26000000000,комунальна,22883,23392178,2020,80,,гмназя237
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,26469.2,NaN,26000000000,комунальна,23029,23392149,2020,80,,середнязагальноосвтняшкола3ступ195менвкудряшов...
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,10294.196,NaN,26000000000,комунальна,23280,26187881,2020,80,,спецалзованашкола3ступ73зпоглибленимвивченямук...
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,13412.1,NaN,26000000000,комунальна,23154,22878447,2020,80,,спецалзованашкола131мкив


## MON 2023

In [31]:
#load data
schools_data = pd.read_excel('./school_data/Schools_02.08.2023 16-56.xlsx', dtype = str)
schools_data

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Код ЄДРПОУ,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,...,Телефон,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,(0800)330810,NaN,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)5264358,(044)5262002,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,"(044)234-50-55, (044)235-60-63",(044)234-50-55,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)259-31-60,(044)525-30-31,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)2584242,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,(046)422-43-40,NaN,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,(0462)687733,NaN,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,(04653)27571,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,(046)5924644,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90


In [32]:
schools_data.columns

Index(['Повна назва', 'Код в ЄДЕБО', 'ОУО підтвердив дані', 'Скорочена назва',
       'Код ЄДРПОУ', 'Статус', 'Тип закладу', 'Форма власності', 'Код КОАТУУ',
       'Регіон', 'Населений пункт', 'Місцезнаходження (юридична адреса)',
       'Головний заклад',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Телефон', 'Факс', 'Електронна пошта', 'Веб-сайт', 'Керівник',
       'Опорний / Філія', 'Сільський', 'Гірський', 'Інтернат',
       'Ліцензовані обсяги'],
      dtype='object')

In [33]:
# create new with id
schools_data_id = schools_data.copy()
create_id(schools_data_id, 'Повна назва', 'id')

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Код ЄДРПОУ,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,NaN,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)5262002,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236мкив
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)234-50-55,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87мкив
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)525-30-31,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150мкив
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,NaN,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1,яблунвськийнавчальновиховнийкомплексзагальноос...
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,NaN,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN,ягднянськафляопорногозакладуколичвськазагально...
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ступборзнянськач...
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ступсеменвськагмназ2семенвськаче...


In [34]:
schools_data_id.rename(columns={'Код КОАТУУ':'KOATUU_2020', 'Код ЄДРПОУ':'EDRPOU'}, inplace=True)
schools_data_id.columns

Index(['Повна назва', 'Код в ЄДЕБО', 'ОУО підтвердив дані', 'Скорочена назва',
       'EDRPOU', 'Статус', 'Тип закладу', 'Форма власності', 'KOATUU_2020',
       'Регіон', 'Населений пункт', 'Місцезнаходження (юридична адреса)',
       'Головний заклад',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Телефон', 'Факс', 'Електронна пошта', 'Веб-сайт', 'Керівник',
       'Опорний / Філія', 'Сільський', 'Гірський', 'Інтернат',
       'Ліцензовані обсяги', 'id'],
      dtype='object')

In [35]:
schools_data_id[schools_data_id.KOATUU_2020.str.len() != 10]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id


In [36]:
schools_data_id.EDRPOU = schools_data_id.EDRPOU.apply(lambda x: x.zfill(8) if isinstance(x, str) and len(x) >= 6 else None)

In [37]:
schools_data_id[(schools_data_id.EDRPOU.str.len() < 8)&(schools_data_id.EDRPOU.str.len().notna())]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id


In [38]:
schools_data_id.loc[:,'KOATUU_2020_reg'] = schools_data_id.loc[:,'KOATUU_2020'].str[:2]
schools_data_id

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки,80
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236мкив,80
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87мкив,80
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150мкив,80
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1,яблунвськийнавчальновиховнийкомплексзагальноос...,74
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN,ягднянськафляопорногозакладуколичвськазагально...,74
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ступборзнянськач...,74
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ступсеменвськагмназ2семенвськаче...,74


In [39]:
schools_data_id = schools_data_id[schools_data_id.EDRPOU != '02301958']
schools_data_id

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки,80
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236мкив,80
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87мкив,80
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150мкив,80
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1,яблунвськийнавчальновиховнийкомплексзагальноос...,74
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN,ягднянськафляопорногозакладуколичвськазагально...,74
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ступборзнянськач...,74
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ступсеменвськагмназ2семенвськаче...,74


In [40]:
schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg', 'EDRPOU'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
1611,Велимченський ліцей Велимченської сільської ра...,176569,так,Велимченський ліцей,21747814,ліквідовано,ліцей,Комунальна,0724280401,Волинська область,...,NaN,NaN,Директор Павлович Тетяна Володимирівна,ні,ні,ні,ні,NaN,велимченськийлцейвелимченськаволинськаобл,07
1612,Велимченський ліцей Велимченської сільської ра...,143856,так,Велимченський ліцей,21747814,працює,ліцей,Комунальна,0724280401,Волинська область,...,velimche__shcola@ukr.net,http://velimcheshkola.at.ua/,Павлович Тетяна Володимирівна,ні,так,ні,ні,NaN,велимченськийлцейвелимченськаволинськаобл,07
10031,Трикратська ЗОШ І-ІІІ ступенів Олександрівсько...,151531,так,NaN,None,працює,школа,Комунальна,4822085001,Миколаївська область,...,a_grigorivna@ukr.net,https://trykraty.e-schools.info/,Директор Багнюк Віталій Михайлович,ні,так,ні,ні,220,трикратськазагальноосвтняшкола3ступолександрвс...,48
10032,Трикратська ЗОШ І-ІІІ ступенів Олександрівсько...,151533,так,Трикратська ЗОШ І-ІІІ ступенів,None,працює,школа,Комунальна,4822085001,Миколаївська область,...,a_grigorivna@ukr.net,https://trykraty.e-schools.info/,Директор Багнюк Віталій Михайлович,ні,так,ні,ні,220,трикратськазагальноосвтняшкола3ступолександрвс...,48
14862,Подо-Калинівський ЗЗСО І-ІІІ ст,151577,так,NaN,None,працює,школа,Комунальна,6525082501,Херсонська область,...,kalina19802@ukr.net,http://www.kalina1980.ucoz.ua,Директор Маляренко Віктор Олександрович,ні,так,ні,ні,464,подокалинвський3с,65
14863,Подо-Калинівський ЗЗСО І-ІІІ ст,151582,так,NaN,None,працює,школа,Комунальна,6525082501,Херсонська область,...,kalina19802@ukr.net,http://www.kalina1980.ucoz.ua,Директор Маляренко Віктор Олександрович,ні,так,ні,ні,464,подокалинвський3с,65
14909,Старо-Маячківський ЗЗСО І ст,151578,так,NaN,None,працює,початкова школа,Комунальна,6525082502,Херсонська область,...,stara-mayachka@ukr.net,NaN,Директор Несторяк Наталія Іванівна,ні,так,ні,ні,40,старомаячквськийс,65
14910,Старо-Маячківський ЗЗСО І-ст,151583,так,NaN,None,працює,початкова школа,Комунальна,6525082502,Херсонська область,...,stara-mayachka@ukr.net,NaN,Директор Несторяк Наталія Іванівна,ні,так,ні,ні,40,старомаячквськийс,65


In [41]:
schools_data_id = schools_data_id.drop_duplicates(subset=['id', 'KOATUU_2020_reg', 'EDRPOU'], keep='last')

In [42]:
schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
91,"Приватний заклад освіти ""УОЛД СКУЛ КАМПУС А""",151623,так,"ПЗО ""УОЛД СКУЛ КАМПУС А""",42180923,працює,школа,Приватна,8036300000,КИЇВ,...,campus.a@worldschool.com.ua,NaN,Директор Бадаєва Інна Миколаївна,ні,ні,ні,ні,NaN,приватнийзакладосвтиуолдскулкампус,80
254,Технічний ліцей м. Києва,145237,так,Технічний ліцей м. Києва,22876023,працює,ліцей,Комунальна,8036600000,КИЇВ,...,dtl-licej@ukr.ua,http://tl.kyiv.ua/,Директор Андрієнко Анатолій Михайлович,ні,ні,ні,ні,NaN,технчнийлцеймкив,80
268,"Приватний заклад освіти ""УОЛД СКУЛ КАМПУС Б""",176647,так,"ПЗО ""УОЛД СКУЛ КАМПУС Б""",43594721,працює,школа,Приватна,8000000000,КИЇВ,...,NaN,NaN,Директор Давидова Наталія Леонідівна,ні,ні,ні,ні,NaN,приватнийзакладосвтиуолдскулкампус,80
584,Технічний ліцей Шевченківського району м. Києва,149078,так,Технічний ліцей,22881946,працює,ліцей,Комунальна,8039100000,КИЇВ,...,texlyceum@ukr.net,techlyceum.kiev.ua,Директор Ілюшина Олена Олександрівна,ні,ні,ні,ні,844,технчнийлцеймкив,80
4728,Данилівський заклад загальної середньої освіти...,176502,так,Данилівський НВК І-ІІІ ступенів,40760974,працює,навчально-виховний комплекс (об'єднання),Комунальна,2125382601,Закарпатська область,...,danylovo@ukr.net,NaN,Директор Гайович Тетяна Василівна,ні,так,ні,ні,171,данилвський3ступхустськазакарпатськаобл,21
4729,Данилівський заклад загальної середньої освіти...,148256,так,ДАНИЛІВСЬКИЙ ЗЗСО І-ІІІ СТУПЕНІВ,34440724,реорганізовано,навчально-виховний комплекс (об'єднання),Комунальна,2125382601,Закарпатська область,...,danylovo@ukr.net,NaN,Директор Гайович Тетяна Василівна,ні,так,ні,ні,NaN,данилвський3ступхустськазакарпатськаобл,21
6818,Гоголівський ліцей Великодимерської селищної р...,146689,так,Гоголівський ліцей,25299425,працює,ліцей,Комунальна,3221281601,Київська область,...,gogoliv.school@gmail.com,http://gogolivska-zosh.kiev.sch.in.ua/,Директор Дяченко Оксана Іванівна,Опорний заклад,так,ні,ні,NaN,гоголвськийлцейвеликодимерськакивськаобл,32
6819,Гоголівський ліцей Великодимерської селищної р...,147130,так,філія Гоголівського ліцею,25299342,працює,ліцей,Комунальна,3221281601,Київська область,...,gogoliv-school2@ukr.net,http://gogoliv-school2.kiev.sch.in.ua,Завідувач філією Мишкоріз Любов Миколаївна,Філія,так,ні,ні,NaN,гоголвськийлцейвеликодимерськакивськаобл,32
11534,"Комунальний заклад ""Розсошенська гімназія Щерб...",139999,так,Розсошенська гімназія,21071866,працює,гімназія,Комунальна,5324087705,Полтавська область,...,rzsgymn@gmail.com,http://rzsschool.org.ua/,Директор Яловець Євген Олександрович,ні,так,ні,ні,834,розсошенськагмназящербанвськаполтавськаобл,53
11535,"Комунальний заклад ""Розсошенська гімназія Щерб...",151527,так,Розсошенська гімназія,52852882,ліквідовано,гімназія,Комунальна,5324087705,Полтавська область,...,rzsgymn@gmail.com,http://rzsschool.ucoz.ua/,Директор Яловець Євген Олександрович,ні,так,ні,ні,NaN,розсошенськагмназящербанвськаполтавськаобл,53


In [43]:
schools_data_id = schools_data_id[~schools_data_id.EDRPOU.isin(['34440724', '52852882', '58826264', 
                                                                '33329415', '26271178', '27646632', 
                                                                '04393999', '24623237', '25649174',
                                                                '25649172', '21977976', '26213928',
                                                                '06716337', '26291911'])]

In [44]:
schools_data_id.drop(schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist(), inplace=True)

In [50]:
dataset_2020_mon= dataset_2020.merge(schools_data_id[['id', 'KOATUU_2020_reg', 'EDRPOU']], on=['id', 'KOATUU_2020_reg'], how='left')
dataset_2020_mon

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,hub,budgetcode,ownership,DISO,EDRPOU_x,year,KOATUU_2020_reg,EDRPOU_checked,id,EDRPOU_y
0,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,5,117,...,NaN,02553000000,комунальна,7137,25844555,2020,05,,загальноосвтнйнавчальновиховнийкомплекс3ступшк...,NaN
1,Вінницька область,місто,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,8,312,...,NaN,02553000000,комунальна,6936,25844992,2020,05,,кнззагальноосвтняшкола3ступ3козятинськавницькаобл,NaN
2,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,8,530,...,NaN,02553000000,комунальна,7149,25844928,2020,05,,комунальнийнавчальнийзакладзагальноосвтняшкола...,NaN
3,Вінницька область,місто,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,7,465,...,NaN,02553000000,комунальна,7191,25844970,2020,05,,загальноосвтнясередняшкола3ступ5козятинськавни...,NaN
4,Вінницька область,місто,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,4,820,...,NaN,02553000000,комунальна,6976,23101028,2020,05,,комунальнийнавчальнийзакладспецалзованашкола3с...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,NaN,26000000000,комунальна,22883,23392178,2020,80,,гмназя237,23392178
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,NaN,26000000000,комунальна,23029,23392149,2020,80,,середнязагальноосвтняшкола3ступ195менвкудряшов...,23392149
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,NaN,26000000000,комунальна,23280,26187881,2020,80,,спецалзованашкола3ступ73зпоглибленимвивченямук...,26187881
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,NaN,26000000000,комунальна,23154,22878447,2020,80,,спецалзованашкола131мкив,22878447


In [51]:
dataset_2020_mon.loc[(dataset_2020_mon['EDRPOU_y'].notna())&(dataset_2020_mon.EDRPOU_checked == ''), 'EDRPOU_checked'] = dataset_2020_mon.loc[(dataset_2020_mon['EDRPOU_y'].notna())&(dataset_2020_mon.EDRPOU_checked == ''), 'EDRPOU_y']

In [52]:
dataset_2020_mon = dataset_2020_mon.drop(columns=['EDRPOU_y'], axis=1)
dataset_2020_mon = dataset_2020_mon.rename(columns = {'EDRPOU_x':'EDRPOU'})

In [53]:
dataset_2020_mon[dataset_2020_mon['EDRPOU_checked'] != '']

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,EDRPOU_checked,id
13,Вінницька область,місто,Могилів-Подільська міська ТГ,Гімназія № 2 м.Могилева-Подільського Вінницько...,I-III,NaN,51,26,4,713,...,16310,NaN,02558000000,комунальна,11323,43125858,2020,05,43125858,гмназя2ммогилеваподльськоговницькаобл
114,Вінницька область,місто,Турбівська селищна ТГ,Турбівська загальноосвітня школа І-ІІІ ст. №2 ...,I-III,NaN,39,17,3,485,...,11977.156,NaN,02568000000,комунальна,11405,21723820,2020,05,21723820,турбвськазагальноосвтняшкола3ступ2х000dлиповец...
120,Вінницька область,село,Іванівська сільська ТГ,Середня загальноосвітня школа І - ІІІ ступенів...,I-III,NaN,23,18,8,195,...,8302,NaN,02526000000,комунальна,7995,26229088,2020,05,26229088,середнязагальноосвтняшкола3ступсуладвкавницькаобл
124,Вінницька область,село,Літинська селищна ТГ,Середня загальноосвітня школа І-ІІІ ступенів с...,I-III,NaN,17,18,1,112,...,5545.8,NaN,02541000000,комунальна,7988,26223660,2020,05,26223660,середнязагальноосвтняшкола3ступсвч
144,Вінницька область,село,Мурованокуриловецька селищна ТГ,"Навчально виховний комплекс ""Загальноосвітній ...",I-III,NaN,13,12,2,44,...,2823.2,NaN,02559000000,комунальна,6399,26336200,2020,05,26336200,навчальновиховнийкомплексзагальноосвтнйнавчаль...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,25731.415,NaN,26000000000,комунальна,22883,23392178,2020,80,23392178,гмназя237
9230,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,26469.2,NaN,26000000000,комунальна,23029,23392149,2020,80,23392149,середнязагальноосвтняшкола3ступ195менвкудряшов...
9231,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,10294.196,NaN,26000000000,комунальна,23280,26187881,2020,80,26187881,спецалзованашкола3ступ73зпоглибленимвивченямук...
9232,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,13412.1,NaN,26000000000,комунальна,23154,22878447,2020,80,22878447,спецалзованашкола131мкив


# Preparation to merge with schools_edrpou dataset

# Merge with schools

In [ ]:
schools_edrpou = pd.read_csv('./final_tables/schools_edrpou.csv', dtype=str)
schools_edrpou

In [ ]:
school_finance_edrpou = school_finance.merge(schools_edrpou[['id', 'KOATUU_2020_reg', 'EDRPOU']], on=['id', 'KOATUU_2020_reg'], how='left')

In [ ]:
school_finance_edrpou.loc[school_finance_edrpou['EDRPOU_y'].notna(), 'EDRPOU'] = school_finance_edrpou.loc[school_finance_edrpou['EDRPOU_y'].notna(), 'EDRPOU_y']

In [ ]:
school_finance_edrpou = school_finance_edrpou.drop(columns=['EDRPOU_y'], axis=1)
school_finance_edrpou = school_finance_edrpou.rename(columns = {'EDRPOU_x':'EDRPOU_old'})

In [ ]:
school_finance_edrpou[school_finance_edrpou['EDRPOU'].isna()]

In [ ]:
school_finance_edrpou.loc[school_finance_edrpou['EDRPOU'].isna(), 'EDRPOU'] = ''
school_finance_edrpou

In [ ]:
school_finance_edrpou[school_finance_edrpou.EDRPOU == '']

# MON 2023

In [ ]:
schools_ed = merging(school_finance_edrpou, schools_data_id[['KOATUU_2020_reg', 'EDRPOU', 'id']], ['id', 'KOATUU_2020_reg'])

schools_ed 

## short name

In [ ]:
schools_data_id_short = schools_data_id[schools_data_id['Скорочена назва'].notna()]

In [ ]:
create_id(schools_data_id_short, 'Скорочена назва', 'id')

In [ ]:
schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg', 'EDRPOU'], keep=False)]

In [ ]:
schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

In [ ]:
schools_data_id_short.drop(schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist(), inplace=True)

In [ ]:
schools_ed = merging(schools_ed, schools_data_id_short[['KOATUU_2020_reg', 'EDRPOU', 'id']], ['id', 'KOATUU_2020_reg'])
schools_ed 

# youcontrol

In [ ]:
youcontrol =  pd.read_excel('./school_data/Youcontrol_historical_name.xlsx', dtype='str')
youcontrol 

In [ ]:
youcontrol[youcontrol.duplicated(subset=['Назва', 'Код ЄДРПОУ'], keep=False)]

In [ ]:
youcontrol_long = youcontrol.drop_duplicates(subset=['Назва', 'Код ЄДРПОУ'], keep='first')

In [ ]:
create_id(youcontrol_long, 'Назва', 'id')

In [ ]:
youcontrol_long.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)

In [ ]:
youcontrol_long[(youcontrol_long.EDRPOU.str.len() < 8)&(youcontrol_long.EDRPOU.str.len().notna())]

In [ ]:
youcontrol_long[youcontrol_long.duplicated(subset=['id', 'EDRPOU'], keep=False)]

In [ ]:
youcontrol_long = youcontrol_long.drop_duplicates(subset=['id', 'EDRPOU'], keep='first')

In [ ]:
youcontrol_long[youcontrol_long.duplicated(subset=['id'], keep=False)]

In [ ]:
youcontrol_long.drop(youcontrol_long[youcontrol_long.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)

In [ ]:
schools_youcontrol_long = merging(schools_ed, youcontrol_long[['id', 'EDRPOU']], ['id'])
schools_youcontrol_long

## short name

In [ ]:
youcontrol_short = youcontrol[youcontrol['Коротка назва'].notna()]

In [ ]:
youcontrol_short.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)

In [ ]:
create_id(youcontrol_short, 'Коротка назва', 'id')

In [ ]:
youcontrol_short[youcontrol_short.duplicated(subset=['Коротка назва', 'EDRPOU'], keep=False)]

In [ ]:
youcontrol_short = youcontrol_short.drop_duplicates(subset=['Коротка назва', 'EDRPOU'], keep='first')

In [ ]:
youcontrol_short[youcontrol_short.duplicated(subset=['id', 'EDRPOU'], keep=False)]

In [ ]:
youcontrol_short = youcontrol_short.drop_duplicates(subset=['id', 'EDRPOU'], keep='first')

In [ ]:
youcontrol_short[youcontrol_short.duplicated(subset=['id'], keep=False)]

In [ ]:
youcontrol_short.drop(youcontrol_short[youcontrol_short.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)

In [ ]:
schools_youcontrol_short = merging(schools_youcontrol_long, youcontrol_short[['id', 'EDRPOU']], ['id'])
schools_youcontrol_short 

# collected with bot

In [ ]:
data_bot = pd.read_csv('./school_data/data_bot.csv', dtype = str)
data_bot

In [ ]:
data_bot.rename(columns={'edrpou':'EDRPOU'}, inplace=True)

In [ ]:
data_bot[(data_bot.EDRPOU.str.len() < 8)&(data_bot.EDRPOU.str.len().notna())]

In [ ]:
data_bot['EDRPOU']=data_bot['EDRPOU'].str.zfill(8)

In [ ]:
data_bot.drop_duplicates(inplace=True)

In [ ]:
data_bot[data_bot.duplicated(subset=['eoname'], keep=False)]

In [ ]:
data_bot.drop(data_bot[data_bot.duplicated(subset=['eoname'], keep=False)].index.values.tolist(), inplace=True)

In [ ]:
schools_bot = merging(schools_youcontrol_short , data_bot[['eoname', 'EDRPOU']], ['eoname'])
schools_bot

# Gathered by hand

In [ ]:
data_hand = pd.read_csv('./school_data/by_hand.csv',  dtype = str)
data_hand

In [ ]:
data_hand = data_hand[['eoname', 'KOATUU_2020', 'EDRPOU', 'KOATUU_2020_reg']]

In [ ]:
data_hand['temp_eoname'] = data_hand['eoname'].str.replace(r'\r\n|\n', '', regex=True).str.strip()

In [ ]:
data_hand[data_hand.duplicated(subset=['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg'], keep=False)]

In [ ]:
data_hand = data_hand.drop_duplicates(subset=['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg'], keep='first')

In [ ]:
data_hand[data_hand.duplicated(subset=['temp_eoname', 'KOATUU_2020_reg'], keep=False)]

In [ ]:
data_hand = data_hand.drop_duplicates(subset=['temp_eoname', 'KOATUU_2020_reg'], keep='first')

In [ ]:
schools_bot['temp_eoname'] = schools_bot['eoname'].str.replace(r'\r\n|\n', '', regex=True).str.strip()

In [ ]:
schools_final = merging(schools_bot, data_hand[['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg']], ['temp_eoname', 'KOATUU_2020_reg'])
schools_final

In [ ]:
schools_final[schools_final.EDRPOU != schools_final.EDRPOU_old]

In [ ]:
# datasets[2019].loc[datasets[2019]['eoname'] == 'Марянівський навчально-виховний комплекс "ДНЗ-ЗОШ" Іступеня', 'EDRPOU'] = '26224046'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Лобойківський заклад загальної середньої освіти Петриківської районної ради Дніпропетровської області', 'EDRPOU'] = '26460466'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Середня загальноосвітня школа І-ІІ ступенів села Вартиківці Кельменецького району Чернівецької області', 'EDRPOU'] = '31967606'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Рипенська загальноосвітня школа І ступеня Путильської районної ради Путильський район Чернівецька область', 'EDRPOU'] = '25811521'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Рижівська загальноосвітня школа І ступеня села Рижа Путильського району Чернівецької області', 'EDRPOU'] = '25811484'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Великобушинська загальноосвітня школа І-ІІ ступенів Немирівської районної ради Вінницької області', 'EDRPOU'] = '25513410'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Мирненська загальноосвітня школа І ступеня Надвірнянської районної ради Івано-Франківської області', 'EDRPOU'] = '20567542'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Завадівський навчально-виховний комплекс І-ІІ ступенів Турківської районної ради Львівської обасті', 'EDRPOU'] = '22417999'
# datasets[2019].loc[datasets[2019]['eoname'] == 'Херсонський навчально-виховний комплекс "Загальноосвітня школа ІІ ступеня-ліцей журналістики, економіки та правознавства" Херсонської міської ради', 'EDRPOU'] = '24114961'

In [ ]:
# datasets[2019][datasets[2019].duplicated(subset=['EDRPOU'], keep=False)]

Якщо подивитися деякі з цих ЄДРПОУ, бачимо таке:

26224046 це Марянівський навчально-виховний комплекс, а друга школа з цим єдрпоу взагалі немає єдрпоу в реєстрах

02144795 це Головне управління освіти і науки львівської обласної державної адміністрації

25811521 це Сергіївська загальноосвітня школа, а Рипенська загальноосвітня школа І ступеня це філіал, який має свій єдрпоу і т.д.

In [ ]:
# datasets[2020].loc[datasets[2020]['EDRPOU'].apply(len) == 7, 'EDRPOU'] = '0' + datasets[2020].loc[datasets[2020]['EDRPOU'].apply(len) == 7, 'EDRPOU']
# datasets[2020].loc[datasets[2020]['eoname'] == 'Турбівська загальноосвітня школа І-ІІІ ст. №2 _x000D_\nЛиповецької районної ради _x000D_\nВінницької області_x000D_\n_x000D_\n', 'EDRPOU'] = '21723820'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Середня загальноосвітня школа І - ІІІ ступенів села Уладівка Літинський район Вінницька область', 'EDRPOU'] = '26229088'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Середня загальноосвітня школа І-ІІІ ступенів с. Івча', 'EDRPOU'] = '26223660'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Комунальний заклад "Загальноосвітня школа І-ІІІ ступенів № 19 Вінницької міської ради"', 'EDRPOU'] = '26235447'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Тисаашванська загальноосвітня школа І-ІІ ступенів Ужгородської районної ради Закарпатської області', 'EDRPOU'] = '22103539'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Пшеничниківський навчально-виховний комплекс "Загальноосвітня школа І-ІІ ступенів - дошкільний навчальний заклад" Тисменицької районної ради Івано-Франківської області', 'EDRPOU'] = '20563076'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Старобільська загальноосвітня школа ІІ-ІІІ ступенів №4 Старобільської районної ради Луганської області', 'EDRPOU'] = '33199525'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Заклад загальноосвітньої середньої освіти І ступення села Смеречка Старосамбірського району Львівської області', 'EDRPOU'] = '00000000'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Загальноосвітня середня школа І ступеня села Дністрик Старосамбірського району Львівської області', 'EDRPOU'] = '00000000'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Заклад загальної середньої освіти І ступеня села Головецько-Золотнівець Старосамбірського району Львівської області', 'EDRPOU'] = '00000000'
# datasets[2020].loc[datasets[2020]['eoname'] == "Воле - Блажівська середня загальноосвітня школа І ступеня Чукв'янської сільської ради Самбірського району Львівської області", 'EDRPOU'] = '22378609'
# datasets[2020].loc[datasets[2020]['eoname'] == 'Загальноосвітня школа І ступеня села Виноградне', 'EDRPOU'] = '24625450'
# datasets[2020].loc[datasets[2020]['eoname'] == ' Херсонський навчально-виховний комплекс "Загальноосвітня школа ІІ ст-ліцей журналістики, економіки та правознавства" Херсонської міської ради', 'EDRPOU'] = '24114961'

In [ ]:
datasets[2020][datasets[2020].duplicated(subset=['EDRPOU'], keep=False)]

In [ ]:
# concatenated_df = pd.concat(list(datasets.values()), ignore_index=True)
# concatenated_df

In [ ]:
# concatenated_df[concatenated_df['Код ЄДРПОУ'].notna()][concatenated_df[concatenated_df['Код ЄДРПОУ'].notna()]['Код ЄДРПОУ'].apply(len) != 8]

In [ ]:
# concatenated_df[['pupilswithpreschool', 'budgetcode', 'DISO', 'Код ЄДРПОУ']] = concatenated_df[['pupilswithpreschool', 'budgetcode', 'DISO', 'Код ЄДРПОУ']].fillna(-10000)
# concatenated_df = concatenated_df.astype({'teachstuff': 'int64', 'nonteachstuff': 'int64', 'teachstuffretage': 'int64', 'pupilswithpreschool': 'int64', 'budgetcode': 'int64', 'DISO': 'int64', 'Код ЄДРПОУ': 'int64'})
# concatenated_df = concatenated_df.astype({'pupilswithpreschool': 'str', 'budgetcode': 'str', 'DISO': 'str', 'Код ЄДРПОУ': 'str'})
# concatenated_df[['pupilswithpreschool', 'budgetcode', 'DISO', 'Код ЄДРПОУ']] = concatenated_df[['pupilswithpreschool', 'budgetcode', 'DISO', 'Код ЄДРПОУ']].replace('-10000', np.NaN)
# concatenated_df

In [ ]:
# concatenated_df.to_csv('Scools_stats.csv', index=False, encoding='utf-8')